In [1]:
%pylab inline
import pickle
import pprint

import numpy as np
import torch

from ogb.lsc import MAG240MDataset

from matplotlib import pyplot as plt

Populating the interactive namespace from numpy and matplotlib


# Config

In [2]:
root = '/lfs/rambo/1/vid/mag240m'
dataset = MAG240MDataset(root)

# Data

In [3]:
num_nodes_dict = {
    'paper': dataset.num_papers,
    'author': dataset.num_authors,
    'institution': dataset.num_institutions
}

In [4]:
path = f'{dataset.dir}/col_dict.p'
col_dict = pickle.load(open(path, 'rb'))
edge_types = { index: edge_type for index, edge_type in enumerate(list(col_dict.keys())) }

In [5]:
edge_types

{0: 'paper cites paper',
 1: 'author writes paper',
 2: 'paper rev_writes author',
 3: 'author affiliated_with institution',
 4: 'institution rev_affiliated_with author'}

In [6]:
node_types_start = {}
num_nodes = 0
start = 0

for node_type, node_count in num_nodes_dict.items():
    node_types_start[node_type] = start
    start += node_count
    num_nodes += node_count

In [7]:
train_idx = torch.from_numpy(dataset.get_idx_split('train')) + node_types_start['paper']
y_train = torch.from_numpy(dataset.paper_label[train_idx])
y_train_all = np.zeros((num_nodes,))
y_train_all[train_idx] = y_train + 1

# Heterogeneous Random Walks

In [8]:
path = f'{dataset.dir}/random_walks_nodes.pt'
random_walks_nodes = torch.load(path)
path2 = f'{dataset.dir}/random_walks_edge_types.pt'
random_walks_edge_types = torch.load(path2)

In [9]:
number_of_walks = random_walks_nodes.shape[0]
walk_length = random_walks_nodes.shape[1] - 1

In [17]:
labels = torch.from_numpy(y_train_all[torch.flatten(random_walks_nodes)].reshape(-1, walk_length + 1))
edge_types_counter = { edge_type: 0 for edge_type in edge_types.keys() }
metapath_counter_analysis = np.zeros((labels.shape[0], len(edge_types.values())))

In [18]:
walks_limit = 1000000

for i in range(walks_limit):
    start_label = labels[i, 0]
    same_label = labels[i] == start_label
    inds = torch.nonzero(same_label, as_tuple=True)[0]

    if inds.shape[0] == 1:
        if i > 0:
            metapath_counter_analysis[i] = metapath_counter_analysis[i - 1]
        continue
    
    for j in range(inds.shape[0] - 1):
        if random_walks_nodes[i, inds[j]] == random_walks_nodes[i, inds[j+1]]:
            continue
        else:
            between_edge_types = random_walks_edge_types[i, inds[j]:inds[j+1]]        
            between_edge_types = torch.unique_consecutive(between_edge_types)
            
            for edge_type in edge_types.keys():
                if str(edge_type) in str(between_edge_types):
                    edge_types_counter[edge_type] += 1
    
    # update analysis
    normalized_edge_types_counter = { key: value / sum(list(edge_types_counter.values())) for key, value in edge_types_counter.items() }
    keys, values = zip(*normalized_edge_types_counter.items())
    metapath_counter_analysis[i, keys] = values

In [19]:
print(edge_types_counter)
print(metapath_counter_analysis[:walks_limit])

{0: 139233, 1: 113775, 2: 113775, 3: 1879, 4: 1879}
[[0.33333333 0.33333333 0.33333333 0.         0.        ]
 [0.27272727 0.36363636 0.36363636 0.         0.        ]
 [0.33333333 0.33333333 0.33333333 0.         0.        ]
 ...
 [0.37575599 0.30705104 0.30705104 0.00507096 0.00507096]
 [0.37575599 0.30705104 0.30705104 0.00507096 0.00507096]
 [0.37575599 0.30705104 0.30705104 0.00507096 0.00507096]]


In [20]:
path = f'{dataset.dir}/edge_types_counter.p'
path2 = f'{dataset.dir}/metapath_counter_analysis.p'

pickle.dump(edge_types_counter, open(path, 'wb'), protocol=4)
pickle.dump(metapath_counter_analysis[:walks_limit], open(path2, 'wb'), protocol=4)

In [21]:
plt.plot(np.arange(walks_limit), metapath_counter_analysis[:walks_limit], label=edge_types.values())
plt.legend()

In [22]:
metapath_counter_analysis[walks_limit - 1]

array([0.37575599, 0.30705104, 0.30705104, 0.00507096, 0.00507096])